### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.

In [ ]:
import os

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" 

! az deployment group show --name {deployment_name} -g {resource_group_name}

ModuleNotFoundError: No module named 'utils'